<a href="https://colab.research.google.com/github/hui509/Prediction-of-quality-of-Wine/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C%EF%BD%9C%E7%B4%85%E9%85%92%E5%93%81%E8%B3%AA%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **載入 `Red Wine Quality` 資料集**



In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv')
df

# **資料概況**

**1. Red Wine Quality 各欄位定義**

* fixed acidity : 非揮發性酸
* volatile acidity : 揮發性酸
* citric acid : 檸檬酸
* residual sugar : 殘留糖分
* chlorides : 氯化物
* free sulfur dioxide : 游離二氧化硫
* total sulfur dioxide : 總二氧化硫
* density : 密度
* pH : pH值
* sulphates : 硫酸鹽
* alcohol : 酒精濃度
* qulity : 品質 (分數介於0 - 10)


**2. 觀察每個「特徵」欄位（除 quality 外）資料的分佈情況**

* 品質越好的紅酒，檸檬酸(citric acid)、硫酸鹽(sulphates)越高。
* 品質越好的紅酒，揮發性酸度(volatile acidity)、氯化物(chlorides)越少。
* 密度(density)對紅酒品質不具有區別性，皆大約為1。
* pH值對紅酒品質區別性也很小，大約落在3左右。




In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# 創建一個3x4的圖
fig, axes = plt.subplots(3, 4, figsize=(12, 8))

# 將axes攤平為一維矩陣，以利後續遍歷
axes = axes.flatten()

# 定義資料欄位的索引
column_index = 0

# 長條圖呈現各特徵的分布
for i in df.columns:
  if i != 'quality':
    sns.barplot(data=df, x='quality', y=i, palette='coolwarm', ax=axes[column_index])
    axes[column_index].set_title(i)
    column_index += 1

# 刪除最後一個空白圖表
axes[-1].remove()

# 調整子圖之間的間距
plt.tight_layout()

# 輸出圖表
plt.show()

**3. 觀察每個「標籤」欄位（quality）資料的分佈情況**

* 資料中的 quality 介於3-8分之間。
* 獲得5、6分 quality 的紅酒占多數，共有1319筆。


In [ ]:
# 計算不同quality的筆數
df['quality'].value_counts()

In [ ]:
# 計數圖呈現quality的分布
sns.countplot(data=df, x='quality', palette='coolwarm')

**4. 觀察各特徵之間的關係**

* 紅酒品質與酒精濃度(alcohol)有較高的正相關，其次為硫酸鹽(sulphates)和檸檬酸(citric acid)。
* 紅酒品質與揮發性酸度(volatile acidity)有較高的負相關。



In [ ]:
# 計算相關性
cor = df.corr()

# 使用seaborn和matplotlib，視覺化呈現
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
sns.heatmap(cor,annot=True,linewidth=0.5,cmap='coolwarm')
plt.title('Correlation Matrix of Red Wine Quality')
plt.show()

# **資料清理與型態轉換**

In [75]:
# 確認是否有缺失值
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [76]:
# 確認資料型態是否符合模型需求
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


# **模型分析**

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 拆分訓練集與測試集
X = df.drop('quality', axis = 1)
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# 使用隨機樹分類模型
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

# **模型評估**

In [81]:
# 指標判斷模型表現
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00        10
           5       0.72      0.78      0.75       130
           6       0.64      0.71      0.67       132
           7       0.62      0.48      0.54        42
           8       0.00      0.00      0.00         5

    accuracy                           0.67       320
   macro avg       0.33      0.33      0.33       320
weighted avg       0.64      0.67      0.65       320



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [82]:
# 交叉驗證
from sklearn.model_selection import cross_val_score
rfc_eval = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 5)
print(rfc_eval)

[0.65234375 0.69140625 0.6640625  0.69921875 0.75686275]
